# 4. Реализация автоматического дифференцирования. Объект GradientTape

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np

2024-10-02 22:14:43.710187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 22:14:43.877528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 22:14:43.923983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
x = tf.Variable(-2.0)

with tf.GradientTape() as tape:
    y = x**2
    
df = tape.gradient(y, x)
print(df)

tf.Tensor(-4.0, shape=(), dtype=float32)


In [3]:
w = tf.Variable(tf.random.normal((3, 2)))
b = tf.Variable(tf.zeros(2, dtype=tf.float32))
x = tf.Variable([[-2.0, 1.0, 3.0]])

In [4]:
with tf.GradientTape() as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y ** 2)
    
df = tape.gradient(loss, [w, b])
print(df[0], df[1], sep="\n")

tf.Tensor(
[[ 10.114283  -13.703579 ]
 [ -5.0571413   6.8517895]
 [-15.171424   20.555368 ]], shape=(3, 2), dtype=float32)
tf.Tensor([-5.0571413  6.8517895], shape=(2,), dtype=float32)


In [5]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.constant(1.5) #производная для константы не вычисляется

In [6]:
with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2*b
    
df = tape.gradient(f, [x, b])
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
None


In [7]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2*b
    
df = tape.gradient(f, [x, b])
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


In [8]:
x = tf.Variable(0, dtype=tf.float32, trainable=False) #запрет вычисления производной
b = tf.Variable(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2*b
    
df = tape.gradient(f, [x, b])
print(df[0], df[1], sep="\n")

None
tf.Tensor(5.0, shape=(), dtype=float32)


In [9]:
x = tf.Variable(0, dtype=tf.float32, trainable=False) #запрет вычисления производной
b = tf.Variable(1.5) + 1 #производная для константы не вычисляется

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2*b
    
df = tape.gradient(f, [x, b])
print(df[0], df[1], sep="\n")

None
None


In [10]:
x = tf.Variable(0, dtype=tf.float32) 
b = tf.Variable(1.5) 

with tf.GradientTape(watch_accessed_variables=False) as tape: #отключение отслеживания всех вычислений
    f = (x + b) ** 2 + 2*b
    
df = tape.gradient(f, [x, b])
print(df[0], df[1], sep="\n")

None
None


In [11]:
x = tf.Variable(0, dtype=tf.float32) 
b = tf.Variable(1.5) 

with tf.GradientTape(watch_accessed_variables=False) as tape: 
    tape.watch(x) #отслеживание определенной переменной
    f = (x + b) ** 2 + 2*b
    
df = tape.gradient(f, [x, b])
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
None


In [12]:
x = tf.Variable(0, dtype=tf.float32) 
b = tf.Variable(1.5) 

with tf.GradientTape(watch_accessed_variables=False) as tape: 
    tape.watch([x, b]) #отслеживание определенной переменной
    f = (x + b) ** 2 + 2*b
    
df = tape.gradient(f, [x, b])
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


In [13]:
x = tf.Variable(0, dtype=tf.float32) 
b = tf.Variable(1.5) 

with tf.GradientTape(watch_accessed_variables=False) as tape: 
    tape.watch(x) #отслеживание определенной переменной
    y = 2 * x
    f = y * y
    
df = tape.gradient(f, y)
print(df)

tf.Tensor(0.0, shape=(), dtype=float32)


In [14]:
x = tf.Variable(0, dtype=tf.float32) 
b = tf.Variable(1.5) 

with tf.GradientTape(watch_accessed_variables=False) as tape: 
    tape.watch(x) #отслеживание определенной переменной
    y = 2 * x
    f = y * y
    
df = tape.gradient(f, [x, y])
print(df)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>]


In [16]:
x = tf.Variable(0, dtype=tf.float32) 
b = tf.Variable(1.5) 

with tf.GradientTape(watch_accessed_variables=False, persistent=True) as tape: 
    tape.watch(x) #отслеживание определенной переменной
    y = 2 * x
    f = y * y
    
df = tape.gradient(f, y)
df_dx = tape.gradient(f, x)

del tape

print(df, df_dx, sep="\n")

tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)


In [20]:
x = tf.Variable(1.0)

with tf.GradientTape() as tape:
    y = [2.0, 3.0] * x**2


df = tape.gradient(y, x)

print(df)

tf.Tensor(10.0, shape=(), dtype=float32)


In [27]:
x = tf.Variable([1.0, 2.0])

with tf.GradientTape() as tape:
    y = tf.reduce_sum([2.0, 3.0] * x ** 2)


df = tape.gradient(y, x)

print(df)

tf.Tensor([ 4. 12.], shape=(2,), dtype=float32)


In [28]:
x = tf.Variable(1.0)

with tf.GradientTape() as tape:
    if x < 2.0:
        y = tf.reduce_sum([2.0, 3.0] * x ** 2)
    else:
        y = x**2


df = tape.gradient(y, x)

print(df)

tf.Tensor(10.0, shape=(), dtype=float32)


In [32]:
x = tf.Variable(1.0)


with tf.GradientTape() as tape:
    y = 2 * x + 1
    z = y ** 2


df = tape.gradient(z, x)

print(df)

tf.Tensor(12.0, shape=(), dtype=float32)


In [38]:
x = tf.Variable(1.0)

for n in range(2):
    with tf.GradientTape() as tape:
        y = x ** 2 + 2* x

    df = tape.gradient(y, x)

    print(df)
    x.assign_add(1.0)

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


In [40]:
x = tf.Variable(1.0)

with tf.GradientTape() as tape:
    y = tf.constant(2.0) + x * x


df = tape.gradient(y, x)

print(df)

tf.Tensor(2.0, shape=(), dtype=float32)


In [44]:
x = tf.Variable(1, dtype=tf.float32) # или 1.0

with tf.GradientTape() as tape:
    y = x * x


df = tape.gradient(y, x)

print(df)

tf.Tensor(2.0, shape=(), dtype=float32)


In [46]:
x = tf.Variable(1.0)
w = tf.Variable(2.0)

with tf.GradientTape() as tape:
    w.assign_add(x)
    y = w**2


df = tape.gradient(y, x)

print(df)

None


In [47]:
x = tf.Variable(1.0)
w = tf.Variable(2.0)

with tf.GradientTape() as tape:
    w = w + x
    y = w**2


df = tape.gradient(y, x)

print(df)

tf.Tensor(6.0, shape=(), dtype=float32)
